In [ ]:
import logging
from beamlime.kafka.sink import KafkaSink, serialize_dataarray_to_f144
from beamlime import Message, StreamId, StreamKind

instrument = 'bifrost'

# Create logger that prints to stdout
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(
    logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
)
logger.addHandler(handler)

sink = KafkaSink(
    kafka_config={"bootstrap.servers": "localhost:9092"},
    instrument=instrument,
    serializer=serialize_dataarray_to_f144,
    logger=logger,
)

In [ ]:
import time
import scipp as sc

da = sc.DataArray(sc.scalar(1.2), coords={'time': sc.scalar(time.time_ns(), unit='ns')})
msg = Message(value=da, stream=StreamId(kind=StreamKind.LOG, name='detector_angle'))
sink.publish_messages([msg])

In [ ]:
import ipywidgets as widgets
from IPython.display import display


def on_slider_change(change):
    """Callback function that publishes a message when slider value changes"""
    value = change['new']
    da = sc.DataArray(
        sc.scalar(value), coords={'time': sc.scalar(time.time_ns(), unit='ns')}
    )
    msg = Message(value=da, stream=StreamId(kind=StreamKind.LOG, name='detector_angle'))
    sink.publish_messages([msg])
    print(f"Published message with value: {value}")


slider = widgets.FloatSlider(
    value=1.2,
    min=0.0,
    max=10.0,
    step=0.1,
    description='Detector Angle:',
    continuous_update=False,
)

slider.observe(on_slider_change, names='value')
display(slider)